## Example Strip Augmentation Run

### Note:
**Similar Protocol Applies to Patch Augmentation**

### Strip-5:

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os
import numpy as np

# Set a seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

def remove_strip(image, orientation='horizontal', strip_size=5):
    if orientation == 'horizontal':
        # Choose a random horizontal strip to remove
        y = np.random.randint(0, image.shape[0] - strip_size)
        image[y:y+strip_size, :, :] = 0
    else:
        # Choose a random vertical strip to remove
        x = np.random.randint(0, image.shape[1] - strip_size)
        image[:, x:x+strip_size, :] = 0
    return image

def strip_removal(image):  # Only takes one argument
    rand_num = np.random.rand()
    if rand_num < 0.15:
        orientation = 'horizontal'
        image = remove_strip(image, orientation)
    elif rand_num < 0.3:
        orientation = 'vertical'
        image = remove_strip(image, orientation)
    return image  # Return the image

# Load the pre-trained Xception model
base_model = tf.keras.applications.Xception(input_shape=(256, 256, 3),
                                            include_top=False,
                                            weights='imagenet')

# Unfreeze the base model
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

# Defining my ArcFace Function
class ArcFace(layers.Layer):
    def __init__(self, n_classes=300, s=60.0, m=0.5, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizer

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[0][-1], self.n_classes),
                                 initializer='glorot_uniform',
                                 trainable=True,
                                 regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = tf.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        theta = tf.acos(tf.clip_by_value(logits, -1.0 + tf.keras.backend.epsilon(), 1.0 - tf.keras.backend.epsilon()))
        target_logits = tf.cos(theta + self.m)
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'regularizer': self.regularizer,
        })
        return config

# Add new layers
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# Placeholder for label input
y = tf.keras.Input(shape=(300,))  # assuming you have 300 classes
output = ArcFace(n_classes=300)([x, y])
print(output)

# Placeholder for label input
y = tf.keras.Input(shape=(300,))  # assuming you have 300 classes
output = ArcFace(n_classes=300)([x, y])
print(output)

# Define the new model
model = Model(inputs=[base_model.input, y], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Add the patch removal function to your ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    preprocessing_function=strip_removal
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Custom data generator
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, directory, datagen, batch_size=32, target_size=(256, 256), class_mode='categorical'):
        self.generator = datagen.flow_from_directory(directory, target_size=target_size, batch_size=batch_size, class_mode=class_mode)
        
    def __len__(self):
        return len(self.generator)
    
    def __getitem__(self, index):
        x, y = self.generator[index]
        return [x, y], y

# Create custom data generators
train_generator = CustomDataGen('ocular_recognition/train', train_datagen)
val_generator = CustomDataGen('ocular_recognition/validation', val_datagen)

# Set up callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
os.makedirs('trained_models', exist_ok=True)
model_checkpoint = ModelCheckpoint('trained_models/Xception_best_model_ArcFace_Strip5.h5', save_best_only=True)

callbacks = [early_stop, reduce_lr, model_checkpoint]

# Fit the model on the batches generated by datagen.flow()
model.fit(train_generator,
          validation_data=val_generator,
          epochs=100,
          callbacks=callbacks)

KerasTensor(type_spec=TensorSpec(shape=(None, 300), dtype=tf.float32, name=None), name='arc_face/Softmax:0', description="created by layer 'arc_face'")
KerasTensor(type_spec=TensorSpec(shape=(None, 300), dtype=tf.float32, name=None), name='arc_face_1/Softmax:0', description="created by layer 'arc_face_1'")
Found 5076 images belonging to 300 classes.
Found 1621 images belonging to 300 classes.
Epoch 1/100
159/159 [==============================] - 65s 377ms/step - loss: 33.9926 - accuracy: 0.0000e+00 - val_loss: 31.3079 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/100
159/159 [==============================] - 65s 407ms/step - loss: 29.7595 - accuracy: 0.0000e+00 - val_loss: 26.1466 - val_accuracy: 0.0259 - lr: 0.0010
Epoch 3/100
159/159 [==============================] - 61s 380ms/step - loss: 25.2147 - accuracy: 0.0030 - val_loss: 17.9659 - val_accuracy: 0.1518 - lr: 0.0010
Epoch 4/100
159/159 [==============================] - 60s 376ms/step - loss: 20.3566 - accuracy: 0.0203 - va

In [2]:
# Create a data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Custom data generator
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, directory, datagen, batch_size=32, target_size=(256, 256), class_mode='categorical'):
        self.generator = datagen.flow_from_directory(directory, target_size=target_size, batch_size=batch_size, class_mode=class_mode)
        
    def __len__(self):
        return len(self.generator)
    
    def __getitem__(self, index):
        x, y = self.generator[index]
        return [x, y], y

# Load images from the test directory
test_generator = CustomDataGen('ocular_recognition/test', test_datagen)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

Found 1621 images belonging to 300 classes.
51/51 [==============================] - 5s 91ms/step - loss: 5.1478e-09 - accuracy: 1.0000
Test accuracy: 1.0


In [3]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the saved model
# model = load_model('trained_models/Xception_best_model_ArcFace_Strip5.h5', custom_objects={'ArcFace': ArcFace})

# Load and preprocess a single image for prediction
image_path = 'ocular_recognition/test/1141/1141_l_1.png'
img = image.load_img(image_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize pixel values to the range [0, 1]

# Create dummy labels
dummy_labels = np.zeros((1, 300))  # assuming you have 300 classes

# Make predictions
predictions = model.predict([img_array, dummy_labels])

# Get the class label with the highest probability
predicted_class = np.argmax(predictions[0])

# Get the class labels
class_labels = list(test_generator.generator.class_indices.keys())

# Get the predicted class label
predicted_class_label = class_labels[predicted_class]

print(f'Predicted class for the single image: {predicted_class_label}')

1/1 [==============================] - 0s 443ms/step
Predicted class for the single image: 1141
